<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/integrations/GoogleGenerativeAIEmbeddings_pinecone_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Langchain
https://python.langchain.com/v0.1/docs/integrations/text_embedding/google_generative_ai/

#Pinecone
https://docs.pinecone.io/home

# VertexAi
https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versioning#stable-versions-available.md


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -qU pinecone-client langchain-google-genai google-cloud-aiplatform langchain-pinecone langchain langchain-community

In [ ]:
from google.colab import userdata

PINECONE_KEY = userdata.get('PINECONE_KEY')
GOOGLE_API_KEY  = userdata.get('GOOGLE_API_KEY')

In [ ]:
import os
import uuid
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = PINECONE_KEY
if "PINECONE_INDEX_NAME" not in os.environ:
    os.environ["PINECONE_INDEX_NAME"] = "forensic"

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")

In [ ]:
len(vector)

In [ ]:
prompt1 = """
Resumen lo más sintetizado que se pueda con la fecha de creación de este informe, poniendo siempre al principio de dicho resumen la siguiente frase “Informe de fecha”,
seguido del lugar de asistencia médica o el lugar donde se creó el informe, el servicio médico donde se atiende al paciente y terminando estas indicaciones con dos puntos y seguidos.
A continuación redacta un resumen muy sintetizado con los datos del reconocimiento médico, la evolución y posibilidades terapéuticas y de las conclusiones,
relacionando los párrafos entre ellos y separándolos con puntos y seguidos (escribirlo remarcando los diferentes apartados, con vocabulario y tecnicismo médico.
No poner la edad del paciente o de la persona a la que se refiere el informe. Referirse al paciente o a la persona a la que se refiere el informe con la abreviatura Sr. o Sra. delante de su apellido si aparece)."""

name_prompt1="""INFORME MÉDICO DE SÍNTESIS DE INCAPACIDAD PERMANENTE"""
keywords1 =  ["informe","incapacidad", "hospital"]

In [ ]:
prompt2 = """Redacta un resumen muy escueto con la fecha de creación de este informe, poniendo siempre al principio de dicho resumen la siguiente frase “Informe de fecha” seguido de la fecha del informe, seguido del lugar de asistencia médica o el lugar donde se creó el informe, el servicio médico donde se atiende al paciente y terminando estas indicaciones con dos puntos y seguidos. A continuación redacta un resumen muy escueto con los datos de la empresa y de su representante y los datos del trabajador, y otro resumen muy escueto de la descripción de las tareas realizadas (escribirlo sin poner apartados ni puntos aparte pero remarcando los diferentes apartados, escribirlo con vocabulario y tecnicismo médico. No poner la edad del paciente o de la persona a la que se refiere el informe. Referirse al paciente o a la persona a la que se refiere el informe con la abreviatura Sr. o Sra. delante de su apellido si aparece)."""

name_prompt2="""CERTIFICADO TAREAS"""
keywords2 =  ["certificado","tareas", "centro salud"]

In [ ]:
prompt3 = """Resumen lo más sintetizado que se pueda con la fecha de creación de este informe, poniendo siempre al principio de dicho resumen la siguiente frase “Informe de fecha”, seguido del lugar de asistencia médica o el lugar donde se creó el informe, el servicio médico donde se atiende al paciente y terminando estas indicaciones con dos puntos y seguidos. A continuación redacta un resumen muy sintetizado con los datos del resumen clínico, motivo de consulta, evolución y comentario, información de pruebas complementarias, impresión diagnóstica, tratamiento recomendado, destino y recomendación, relacionando los párrafos entre ellos y separándolos con puntos y seguidos (escribirlo remarcando los diferentes apartados, con vocabulario y tecnicismo médico. No poner la edad del paciente o de la persona a la que se refiere el informe. Referirse al paciente o a la persona a la que se refiere el informe con la abreviatura Sr. o Sra. delante de su apellido si aparece).

Resume al máximo que se pueda con la fecha de creación de este informe, poniendo siempre al principio de dicho resumen la siguiente frase “Informe de fecha”, seguido del lugar de asistencia médica o el lugar donde se creó el informe, el servicio médico donde se atiende al paciente y terminando estas indicaciones con dos puntos y seguidos. A continuación redacta un resumen muy sintetizado con los datos del resumen clínico, motivo de consulta, evolución y comentario, información de pruebas complementarias, impresión diagnóstica, tratamiento recomendado, destino y recomendación, relacionando los párrafos entre ellos y separándolos con puntos y seguidos (escribirlo remarcando los diferentes apartados, con vocabulario y tecnicismo médico. No poner la edad del paciente o de la persona a la que se refiere el informe. Referirse al paciente o a la persona a la que se refiere el informe con la abreviatura Sr. o Sra. delante de su apellido si aparece).

Diferenciando y separando cada informe redacta una redacción que sea lo más resumida posible de cada apartado que haya en cada informe con la fecha de creación de este informe, poniendo siempre al principio de dicho resumen la siguiente frase “Informe de fecha”, seguido del lugar de asistencia médica o el lugar donde se creó el informe, el servicio médico donde se atiende al paciente y terminando estas indicaciones con dos puntos y seguidos. A continuación redacta un resumen muy sintetizado con los datos del resumen clínico, motivo de consulta, evolución y comentario, información de pruebas complementarias, impresión diagnóstica, tratamiento recomendado, destino y recomendación (si no aparecen esos datos no escribirlos) relacionando los párrafos entre ellos y separándolos con puntos y seguidos (escribirlo con vocabulario y tecnicismo médico. No poner la edad del paciente o de la persona a la que se refiere el informe. Referirse al paciente o a la persona a la que se refiere el informe con la abreviatura Sr. o Sra. delante de su apellido si aparece).

Escríbelo con el siguiente formato:
1. 17/04/2019 - Hospital Universitario Virgen Macarena (Sevilla): Consulta por sospecha de síndrome de apnea e hipopnea durante el sueño. Se recomienda pérdida de peso.
2. 23/04/2019 - SUAP de Brenes (Sevilla): Alta de urgencias por hernia de línea media. Se trató con analgesia y se recomendó acudir a urgencias del hospital.
"""

name_prompt3="""INFORME DE CONSULTA DE REVISIÓN"""
keywords3 =  ["informe","revision", "hospital"]

In [ ]:
vectors=[]
for name_prompt, prompt  in zip([name_prompt1,name_prompt2,name_prompt3],[prompt1,prompt2,prompt3]):

  vectors.append(embeddings.embed_query(name_prompt.strip() + "\n" + prompt.strip()))

In [ ]:
len(vectors[0])

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_KEY)
index = pc.Index("forensic")


In [ ]:
uuid.uuid4()

In [ ]:
for name_prompt, prompt, keywords,vector  in zip([name_prompt1,name_prompt2,name_prompt3],[prompt1,prompt2,prompt3],[keywords1,keywords2,keywords3],vectors):

  index.upsert(
    vectors=[
        {
            "id": str(uuid.uuid4()),
            "values": vector,
            "metadata": {"prompt": prompt.strip(), "name_prompt": name_prompt.strip(), "keywords": keywords}
        }
    ],
    namespace= "prompts"
)




In [ ]:
v_query = embeddings.embed_query("Informe medico de incapacidad permanente de un hospital")

In [ ]:
len(v_query)

In [ ]:
r = index.query(
    namespace="prompts",
    vector=list(v_query),
    top_k=1, include_metadata=True,)

In [ ]:
r

In [ ]:
r = index.query(
    namespace="prompts",
    vector=list(v_query),
    top_k=1, include_metadata=True,
    filter={"$or": [{"keywords": "car"}, {"keywords":"task"}]})


In [ ]:
r